In [1]:
"""
To create the function that will do the filtering 
function for the AutoProofreadNeuron table


"""

'\nTo create the function that will do the filtering \nfunction for the AutoProofreadNeuron table\n\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

In [4]:
import neuron_utils as nru
import neuron_visualizations as nviz

INFO - 2021-02-16 00:26:28,198 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-16 00:26:28,201 - settings - Setting database.user to celiib
INFO - 2021-02-16 00:26:28,202 - settings - Setting database.password to newceliipass
INFO - 2021-02-16 00:26:28,206 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-16 00:26:28,207 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-16 00:26:28,222 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-16 00:26:28,538 - settings - Setting enable_python_native_blobs to True


In [5]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-16 00:26:28,657 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-16 00:26:28,980 - settings - Setting enable_python_native_blobs to True


# Getting the Neuron

In [6]:
curr_segment_id = 864691135012431606
curr_split_index = 0

In [7]:
# neuron_obj = du.decomposition_with_spine_recalculation(segment_id=curr_segment_id,
#                                          split_index=curr_split_index)

In [8]:
# nviz.visualize_neuron(neuron_obj,
#                      limb_branch_dict="all")

# Function that will do proofreading

In [9]:
key = dict(segment_id=curr_segment_id)
verbose=True
proof_version = 0
compute_synapse_to_soma_skeletal_distance = True

In [11]:
import classification_utils as clu
from pykdtree.kdtree import KDTree
import proofreading_utils as pru
import neuron_utils as nru
import time
import trimesh_utils as tu
import datajoint_utils as du
import numpy as np

def proofreading_table_processing(key,
                                  proof_version,
                                  compute_synapse_to_soma_skeletal_distance=True,
                                 verbose=True,)
    """
    Purpose: To do the proofreading and synapse filtering 
    for the datajoint tables

    """

    # 1) Pull Down All of the Neurons
    segment_id = key["segment_id"]

    print(f"\n\n------- AutoProofreadNeuron {segment_id}  ----------")

    neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

    if verbose:
        print(f"Number of Neurons found ={len(neuron_objs)}")


    # 2)  ----- Pre-work ------

    nucleus_ids,nucleus_centers = du.segment_to_nuclei(segment_id)

    if verbose:
        print(f"Number of Corresponding Nuclei = {len(nucleus_ids)}")
        print(f"nucleus_ids = {nucleus_ids}")
        print(f"nucleus_centers = {nucleus_centers}")



    original_mesh = du.fetch_segment_id_mesh(segment_id)
    original_mesh_kdtree = KDTree(original_mesh.triangles_center)



    # 3) ----- Iterate through all of the Neurons and Proofread --------

    # lists to help save stats until write to ProofreadStats Table
    filtering_info_list = []
    synapse_stats_list = []
    total_error_synapse_ids_list = []


    for split_index,neuron_obj_pre_split in zip(neuron_split_idxs,neuron_objs):

        whole_pass_time = time.time()

        if verbose:
            print(f"\n-----Working on Neuron Split {split_index}-----")



        neuron_obj = neuron_obj_pre_split
    #             if neuron_obj_pre_split.n_error_limbs > 0:
    #                 if verbose:
    #                     print(f"   ---> Pre-work: Splitting Neuron Limbs Because still error limbs exist--- ")
    #                 neuron_objs_split = pru.split_neuron(neuron_obj_pre_split,
    #                                              verbose=False)

    #                 if len(neuron_objs_split) > 1:
    #                     raise Exception(f"After splitting the neuron there were more than 1: {neuron_objs_split}")

    #                 neuron_obj= neuron_objs_split[0]
    #             else:
    #                 neuron_obj = neuron_obj_pre_split



        # Part A: Proofreading the Neuron
        if verbose:
            print(f"\n   --> Part A: Proofreading the Neuron ----")


    #     nviz.visualize_neuron(neuron_obj,
    #                       limb_branch_dict="all")



        output_dict= pru.proofread_neuron(neuron_obj,
                            plot_limb_branch_filter_with_disconnect_effect=False,
                            plot_final_filtered_neuron=False,
                            verbose=True)

        filtered_neuron = output_dict["filtered_neuron"]
        cell_type_info = output_dict["cell_type_info"]
        filtering_info = output_dict["filtering_info"]





        # Part B: Getting Soma Centers and Matching To Nuclei
        if verbose:
            print(f"\n\n    --> Part B: Getting Soma Centers and Matching To Nuclei ----")


        winning_nucleus_id, nucleus_info = nru.pair_neuron_obj_to_nuclei(filtered_neuron,
                                 "S0",
                                  nucleus_ids,
                                  nucleus_centers,
                                 nuclei_distance_threshold = 15000,
                                  return_matching_info = True,
                                 verbose=True)

        if verbose:
            print(f"nucleus_info = {nucleus_info}")
            print(f"winning_nucleus_id = {winning_nucleus_id}")






        # Part C: Getting the Faces of the Original Mesh
        if verbose:
            print(f"\n\n    --> Part C: Getting the Faces of the Original Mesh ----")

        original_mesh_faces = tu.original_mesh_faces_map(original_mesh,
                                                    filtered_neuron.mesh,
                                                    exact_match=True,
                                                    original_mesh_kdtree=original_mesh_kdtree)

        original_mesh_faces_file = du.save_proofread_faces(original_mesh_faces,
                                                          segment_id=segment_id,
                                                          split_index=split_index,
                                    file_name_ending=f"proofv{proof_version}_neuron")




    #     nviz.plot_objects(recovered_mesh)


        # Part C.2: Getting the axon information to use for the synapse erroring
        axon_limb_branch_dict = clu.axon_limb_branch_dict(filtered_neuron)

        axon_skeletal_length = nru.sum_feature_over_limb_branch_dict(filtered_neuron,
                                         limb_branch_dict=axon_limb_branch_dict,
                                         feature="skeletal_length")

        axon_mesh_area = nru.sum_feature_over_limb_branch_dict(filtered_neuron,
                                             limb_branch_dict=axon_limb_branch_dict,
                                             feature="area")

        axon_face_labels = clu.axon_faces_from_labels_on_original_mesh(filtered_neuron,
                                               original_mesh=original_mesh,
                                               original_mesh_kdtree=original_mesh_kdtree,
                                                plot_axon=False,
                                               verbose=False,)

        original_mesh_faces_file_axon = du.save_proofread_faces(original_mesh_faces,
                                                          segment_id=segment_id,
                                                          split_index=split_index,
                                                    file_name_ending=f"proofv{proof_version}_axon")

        # Part D: Getting the Synapse Information
        if verbose:
            print(f"\n\n    --> Part D: Getting the Synapse Information ----")


        (keys_to_write_without_version,
         synapse_stats,
         total_error_synapse_ids
        ) = pru.synapse_filtering(filtered_neuron,
                        split_index,
                        nucleus_id=winning_nucleus_id,
                        segment_id=None,
                        return_synapse_filter_info = True,
                        return_synapse_center_data = False,
                        return_error_synapse_ids = True,
                       return_valid_synapse_centers=compute_synapse_to_soma_skeletal_distance,
                        mapping_threshold = 500,
                          plot_synapses=False,
                        verbose = True,
                        original_mesh_method = True,
                        original_mesh = original_mesh,
                        original_mesh_kdtree = original_mesh_kdtree,
                        valid_faces_on_original_mesh=original_mesh_faces, 
                        axon_faces_on_original_mesh=axon_face_labels,

                        )

        # -- 2/15: Will calculate the synapse distances ---- #

        keys_to_write = [dict(k,ver=key["ver"])
                                     for k in keys_to_write_without_version]



        soma_x,soma_y,soma_z = nru.soma_centers(filtered_neuron,
                                           soma_name="S0",
                                           voxel_adjustment=True)





        #7) Creating the dictionary to insert into the AutoProofreadNeuron
        new_key = dict(key,
                       split_index = split_index,
                       proof_version = proof_version,

                       multiplicity = len(neuron_objs),

                       # -------- Important Excitatory Inhibitory Classfication ------- #
                    cell_type_predicted = cell_type_info["inh_exc_class"],
                    spine_category=cell_type_info["spine_category"],

                    n_axons=cell_type_info["n_axons"],
                    n_apicals=cell_type_info["n_axons"],




                    # ----- Soma Information ----#
                    nucleus_id         = nucleus_info["nuclei_id"],
                    nuclei_distance      = np.round(nucleus_info["nuclei_distance"],2),
                    n_nuclei_in_radius   = nucleus_info["n_nuclei_in_radius"],
                    n_nuclei_in_bbox     = nucleus_info["n_nuclei_in_bbox"],

                    soma_x           = soma_x,
                    soma_y           =soma_y,
                    soma_z           =soma_z,

                    # ---------- Mesh Faces ------ #
                    mesh_faces = original_mesh_faces_file,


                    # ------------- The Regular Neuron Information (will be computed in the stats dict) ----------------- #



                       # ------ Information Used For Excitatory Inhibitory Classification -------- 
                    axon_angle_maximum=cell_type_info["axon_angle_maximum"],
                    spine_density_classifier=cell_type_info["neuron_spine_density"],
                    n_branches_processed=cell_type_info["n_branches_processed"],
                    skeletal_length_processed=cell_type_info["skeletal_length_processed"],
                    n_branches_in_search_radius=cell_type_info["n_branches_in_search_radius"],
                    skeletal_length_in_search_radius=cell_type_info["skeletal_length_in_search_radius"],




                       run_time=np.round(time.time() - whole_pass_time,4)
                      )







        stats_dict = filtered_neuron.neuron_stats()
        new_key.update(stats_dict)


#     # ------ Writing the Data To the Tables ----- #
#     SynapseProofread.insert(keys_to_write,skip_duplicates=True)

#     self.insert1(new_key,skip_duplicates=True,allow_direct_insert=True)



#     #saving following information for later processing:
#     filtering_info_list.append(filtering_info)
#     synapse_stats_list.append(synapse_stats)
#     total_error_synapse_ids_list.append(total_error_synapse_ids)



# # Once have inserted all the new neurons need to compute the stats
# if verbose:
#     print("Computing the overall stats")

# overall_syn_error_rates = pru.calculate_error_rate(total_error_synapse_ids_list,
#                 synapse_stats_list,
#                 verbose=True)


# # Final Part: Create the stats table entries and insert

# proofread_stats_entries = []

# stats_to_make_sure_in_proofread_stats = [

#  'axon_on_dendrite_merges_error_area',
#  'axon_on_dendrite_merges_error_length',
#  'low_branch_clusters_error_area',
#  'low_branch_clusters_error_length',
#  'dendrite_on_axon_merges_error_area',
#  'dendrite_on_axon_merges_error_length',
#  'double_back_and_width_change_error_area',
#  'double_back_and_width_change_error_length',
#  'crossovers_error_area',
#  'crossovers_error_length',
#  'high_degree_coordinates_error_area',
#  'high_degree_coordinates_error_length',
# ]


# for sp_idx,split_index in enumerate(neuron_split_idxs):
#     synapse_stats = synapse_stats_list[sp_idx]
#     filtering_info = filtering_info_list[sp_idx]

#     curr_key = dict(key,
#                    split_index = split_index,
#                    proof_version = proof_version,


#                     # ------------ For local valid synapses to that split_index
#                     n_valid_syn_presyn_for_split=synapse_stats["n_valid_syn_presyn"],
#                     n_valid_syn_postsyn_for_split=synapse_stats["n_valid_syn_postsyn"],
#                     n_presyn_error_syn_non_axon = synapse_stats["n_errored_syn_presyn_non_axon"],



#                    )


#     for s in stats_to_make_sure_in_proofread_stats:
#         if s not in filtering_info.keys():
#             curr_key[s] = None

#     filter_key = {k:np.round(v,2) for k,v in filtering_info.items() if "area" in k or "length" in k}
#     curr_key.update(filter_key)
#     curr_key.update(overall_syn_error_rates)

#     proofread_stats_entries.append(curr_key)




------- AutoProofreadNeuron 864691135012431606  ----------
Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1
Number of Neurons found =1
Number of Corresponding Nuclei = 1
nucleus_ids = [553798]
nucleus_centers = [[1271552  476288  971040]]

-----Working on Neuron Split 0-----

   --> Part A: Proofreading the Neuron ----
---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 19.77967381477356


 ------ Part C: Inhibitory Excitatory Classification ---- 


spine_category = densely_spined
Did not pass final filters to continuing
Inhibitory Excitatory Classification = 8.86283564567566

 -- Cell Type Classification Results --
inh_exc_class=excitatory
spine_category=densely_spined
axon_angles={5: {0: 166.80806361041965}}
n_axons=1
n_apicals=1
neuron_spine_density=0.0007039278693363126
n_branches_processe

curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L0 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L1 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L2 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L3 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = [2784]
curr_high_degree_coordinates = [[1263760.  488791.  980440.]]
high_degree_branch_groups = [[2, 3, 4, 5]]
branch 2 did not have one of the following in their labels : ['axon', 'axon-like']

--- Working on Limb L4 ---
edges_to_create = []
edges_to_create

In [41]:
key["ver"] = 30
curr_output = pru.proofreading_table_processing(key,
                                  proof_version=2,
                                  compute_synapse_to_soma_skeletal_distance=True,
                                 verbose=True,)



------- AutoProofreadNeuron 864691135012431606  ----------
Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1
Number of Neurons found =1
Number of Corresponding Nuclei = 1
nucleus_ids = [553798]
nucleus_centers = [[1271552  476288  971040]]

-----Working on Neuron Split 0-----

   --> Part A: Proofreading the Neuron ----
---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


Axon Classification = 20.526360273361206


 ------ Part C: Inhibitory Excitatory Classification ---- 


spine_category = densely_spined
Did not pass final filters to continuing
Inhibitory Excitatory Classification = 9.175243377685547

 -- Cell Type Classification Results --
inh_exc_class=excitatory
spine_category=densely_spined
axon_angles={5: {0: 166.80806361041965}}
n_axons=1
n_apicals=1
neuron_spine_density=0.0007039278693363126
n_branches_proces

curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L0 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L1 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L2 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = []
high_degree_branch_groups = []

--- Working on Limb L3 ---
edges_to_create = []
edges_to_create = []
curr_high_degree_nodes for get_nodes_greater_or_equal_degree_k = [2784]
curr_high_degree_coordinates = [[1263760.  488791.  980440.]]
high_degree_branch_groups = [[2, 3, 4, 5]]
branch 2 did not have one of the following in their labels : ['axon', 'axon-like']

--- Working on Limb L4 ---
edges_to_create = []
edges_to_create

In [43]:
curr_output.keys()

dict_keys(['AutoProofreadSynapse_keys', 'AutoProofreadNeurons_keys', 'filtering_info_list', 'synapse_stats_list', 'total_error_synapse_ids_list', 'neuron_mesh_list', 'axon_mesh_list'])

In [44]:
AutoProofreadSynapse_keys = curr_output["AutoProofreadSynapse_keys"]
AutoProofreadNeurons_keys = curr_output["AutoProofreadNeurons_keys"]
filtering_info_list = curr_output["filtering_info_list"]
synapse_stats_list = curr_output["synapse_stats_list"]
total_error_synapse_ids_list = curr_output["total_error_synapse_ids_list"]
neuron_mesh_list = curr_output["neuron_mesh_list"]
axon_mesh_list = curr_output["axon_mesh_list"]

In [45]:
axon_mesh_list

[PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691135012431606_0_proofv2_axon_proofread.pbz2')]

In [46]:
synapse_stats = synapse_stats_list[0]
synapse_stats

{'n_valid_syn_presyn': 25,
 'n_errored_syn_presyn': 14,
 'n_errored_syn_presyn_non_axon': 12,
 'n_valid_syn_postsyn': 349,
 'n_errored_syn_postsyn': 0}

In [27]:
import neuron
filtered_neuron = neuron.Neuron(filtered_neuron)

Recieved another instance of Neuron class in init -- so just copying data


In [30]:
nru.clear_all_branch_labels(filtered_neuron)

In [21]:
# Part D: Getting the Synapse Information
if verbose:
    print(f"\n\n    --> Part D: Getting the Synapse Information ----")


(keys_to_write,
 synapse_stats,
 total_error_synapse_ids
) = pru.synapse_filtering(filtered_neuron,
                split_index,
                nucleus_id=winning_nucleus_id,
                segment_id=None,
                return_synapse_filter_info = True,
                return_synapse_center_data = False,
                return_error_synapse_ids = True,
               return_valid_synapse_centers=compute_synapse_to_soma_skeletal_distance,
                mapping_threshold = 500,
                  plot_synapses=False,
                verbose = True,
                original_mesh_method = True,
                original_mesh = original_mesh,
                original_mesh_kdtree = original_mesh_kdtree,
                valid_faces_on_original_mesh=original_mesh_faces, 
                axon_faces_on_original_mesh=axon_face_labels,

                )

# # -- 2/15: Will calculate the synapse distances ---- #





# soma_x,soma_y,soma_z = nru.soma_centers(filtered_neuron,
#                                    soma_name="S0",
#                                    voxel_adjustment=True)





# #7) Creating the dictionary to insert into the AutoProofreadNeuron
# new_key = dict(key,
#                split_index = split_index,
#                proof_version = proof_version,

#                multiplicity = len(neuron_objs),

#                # -------- Important Excitatory Inhibitory Classfication ------- #
#             cell_type_predicted = cell_type_info["inh_exc_class"],
#             spine_category=cell_type_info["spine_category"],

#             n_axons=cell_type_info["n_axons"],
#             n_apicals=cell_type_info["n_axons"],




#             # ----- Soma Information ----#
#             nucleus_id         = nucleus_info["nuclei_id"],
#             nuclei_distance      = np.round(nucleus_info["nuclei_distance"],2),
#             n_nuclei_in_radius   = nucleus_info["n_nuclei_in_radius"],
#             n_nuclei_in_bbox     = nucleus_info["n_nuclei_in_bbox"],

#             soma_x           = soma_x,
#             soma_y           =soma_y,
#             soma_z           =soma_z,

#             # ---------- Mesh Faces ------ #
#             mesh_faces = original_mesh_faces_file,


#             # ------------- The Regular Neuron Information (will be computed in the stats dict) ----------------- #



#                # ------ Information Used For Excitatory Inhibitory Classification -------- 
#             axon_angle_maximum=cell_type_info["axon_angle_maximum"],
#             spine_density_classifier=cell_type_info["neuron_spine_density"],
#             n_branches_processed=cell_type_info["n_branches_processed"],
#             skeletal_length_processed=cell_type_info["skeletal_length_processed"],
#             n_branches_in_search_radius=cell_type_info["n_branches_in_search_radius"],
#             skeletal_length_in_search_radius=cell_type_info["skeletal_length_in_search_radius"],




#                run_time=np.round(time.time() - whole_pass_time,4)
#               )







# stats_dict = filtered_neuron.neuron_stats()
# new_key.update(stats_dict)



    --> Part D: Getting the Synapse Information ----
Apply the presyn non_error
For presyn: # valid synapses = 25, # error synapses  = 14, # error presyns = 12
Computing the distance to soma for the synapses
For postsyn: # valid synapses = 349, # error synapses  = 0, # error presyns = 0
Computing the distance to soma for the synapses


In [22]:
keys_to_write[0]

{'synapse_id': 346660423,
 'synapse_type': 'presyn',
 'nucleus_id': 553798,
 'segment_id': 864691135012431606,
 'split_index': 0,
 'skeletal_distance_to_soma': 412.54,
 'syn_center': array([1147376,  787064,  904120])}

In [19]:
synapse_dict_no_path = [k for k in keys_to_write if k["skeletal_distance_to_soma"] == -1]
centers_to_plot = np.array([k["syn_center"] for k in synapse_dict_no_path])
centers_to_plot

array([[1267216,  479936,  972040],
       [1267500,  476940,  976120],
       [1267424,  480608,  972200],
       [1269300,  482876,  972320],
       [1273304,  480880,  974880],
       [1273156,  482000,  973600],
       [1275160,  473596,  973320],
       [1266288,  481528,  968880],
       [1266392,  480176,  969400],
       [1267016,  477764,  970200],
       [1267064,  475632,  969200],
       [1267296,  477608,  967960],
       [1267496,  482200,  970320],
       [1267472,  484040,  967600],
       [1267752,  473864,  968280],
       [1268368,  478456,  965920],
       [1268456,  472192,  967640],
       [1268812,  479308,  965760],
       [1269008,  471456,  966400],
       [1269808,  472120,  965840],
       [1271808,  476988,  964560],
       [1272120,  479536,  964160],
       [1273120,  483160,  971080],
       [1274904,  473952,  969200],
       [1275152,  482176,  971160],
       [1269696,  470596,  965840],
       [1270624,  470024,  971520],
       [1271400,  469820,  9

In [16]:
nviz.plot_objects(filtered_neuron.mesh,
                 scatters=[centers_to_plot],
                 scatters_colors="red",
                 scatter_size=0.2)